## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

## Code

In [1]:
import glob
import numpy as np
import scipy as sp
import scipy.io
import matplotlib.pyplot as plt

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold

### Troika dataset functions

In [4]:
def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

In [5]:
def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

### Auxiliar functions

In [148]:
def FourierTransform(signal, fs, factor=6):
    """
    Computes the frequencies and the magnitudes of the Fourier Transform
    over a signal
    
    Args:
        signal: (np.array) the signal to be transformed
        fs: (number) sampling rate
    
    Returns:
        frequencies and magnitudes of the signal
    """
    window_length = factor * len(signal)
    
    frequencies = np.fft.rfftfreq(window_length, 1 / fs)
    fft = np.abs(np.fft.rfft(signal, window_length))
    
    return frequencies, fft

In [73]:
def ButterworthFilter(signal, pass_band=(40/60, 240/60), fs=125):
    """
    Butterworth filter algorithm.
    
    Returns:
        Bandpass filtered signal
    """          
    b, a = sp.signal.butter(2, pass_band, btype='bandpass', fs = fs)
    return sp.signal.filtfilt(b, a, signal)

In [138]:
def filter_signal(signal):
    signal[signal <= 40/60] = 0
    signal[signal >= 240/60] = 0
    return signal

In [139]:
def Featurize(ppg, accx, accy, accz, fs):
    """
    Featurization of the signal
    
    Args:
        ppg: (np.array) photoplethysmography signal
        accx: (np.array) x-channel of the accelerometer signal
        accy: (np.array) y-channel of the accelerometer signal
        accz: (np.array) z-channel of the accelerometer signal
        fs: (number) sampling rate of the accelerometer signal
    
    Returns:
        n-tuple of PPG and ACC features
    """
    
    # Compute accelerator value with its three channels
    acc = np.sqrt(np.sum(np.array([accx, accy, accz]) ** 2, axis=0))
    
    # Fourier Transform
    ppg_freqs, ppg_fft = FourierTransform(ppg, fs)
    acc_freqs, acc_fft = FourierTransform(acc, fs)

    # Filter ffts
    ppg_fft = filter_signal(ppg_fft)
    acc_fft = filter_signal(acc_fft)
    
    # Features
    ppg_mean_freq = np.mean(ppg_freqs)
    ppg_mean_fft = np.mean(ppg_fft)
    ppg_max_freq = ppg_freqs[np.argmax(ppg_fft)]
    acc_mean_freq = np.mean(acc_freqs)
    acc_mean_fft = np.mean(acc_fft)
    acc_max_freq = acc_freqs[np.argmax(acc_fft)]
    
    return (ppg_mean_freq, 
            ppg_mean_fft, 
            ppg_max_freq,
            acc_mean_freq,
            acc_mean_fft,
            acc_max_freq,
           )

In [8]:
def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

### Classes

In [74]:
class DataProcessor:
    def __init__(self, data_fls, ref_fls, fs=100, window_length=8, window_shift=2):
        # Data
        self.data_fls = data_fls
        self.ref_fls = ref_fls
        # Hyperparameters
        self.fs = fs
        self.window_length = window_length
        self.window_shift = window_shift
    
    def __len__(self):
        return len(self.data_fls)

    def __getitem__(self, idx):
        # Lists to store features, labels and signals
        features, labels, signals = [], [], []
        
        # Extract data
        ppg, accx, accy, accz = LoadTroikaDataFile(self.data_fls[idx])
        bpms = sp.io.loadmat(self.ref_fls[idx])['BPM0'][:, 0]

        # Process data
        total_windows = min(len(ppg), len(bpms))
        left_indexes = np.arange(total_windows, dtype=int) * self.fs * self.window_shift
        right_indexes = left_indexes + self.fs * self.window_length
        
        # Iterate over windows
        for i, (left, right) in enumerate(zip(left_indexes, right_indexes)):
            # Extract portions
            ppg_win = ButterworthFilter(ppg[left:right])
            accx_win = ButterworthFilter(accx[left:right])
            accy_win = ButterworthFilter(accy[left:right])
            accz_win = ButterworthFilter(accz[left:right])
            # Save features, labels and signals
            features.append(Featurize(ppg_win, accx_win, accy_win, accz_win, self.fs))
            labels.append(bpms[i])
            signals.append([ppg_win, accx_win, accy_win, accz_win])

        return features, labels, signals, (ppg, accx, accy, accz)

In [166]:
class Estimator():
    """
    Top-level class for the estimator model
    
    Generates a RandomForestRegressor model and provides functions to interact with it.
    
    """
    def __init__(self, processor, n_estimators=100, max_depth=10):
        # Parameters
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        
        # Model
        self.model = RandomForestRegressor()
        
        # Hyperparameters
        self.processor = processor
    
    def train(self):
        # Generate features
        features, labels, _ = self.generate_features()
        # Split dataset
        X_train, _, y_train, _ = train_test_split(features, labels, test_size=0.2)
        # Train model
        self.model.fit(X_train, y_train)
    
    def predict(self, X):
        return self.model.predict(X)
            
    def generate_features(self):
        features, labels, signals = [], [], []

        for i in range(len(self.processor)):
            _features, _labels, _signals, _ = self.processor[i]
            features.extend(_features)
            labels.extend(_labels)
            signals.extend(_signals)

        return np.array(features), np.array(labels), np.array(signals)

### Main functions

In [167]:
def RunPulseRateAlgorithm(data_fls, ref_fls):
    """
    Runs the pulse rate algorithm on the given data files, trains a model and evaluates its
    performance on the data computing the MAE and the confidence metric.
    """
    # Generate classes
    processor = DataProcessor(data_fls, ref_fls)
    model = Estimator(processor=processor)
    # Train model
    model.train()

    # Predict
    errors, confidences = [], []
    for i in range(len(processor)):
        # Predict
        features, labels, signals, _ = processor[i]
        predictions = model.predict(features)

        _errors, _confidences = [], []
        for i in range(predictions.shape[0]):
            ppg = ButterworthFilter(signals[i][0])

            freqs, fft = FourierTransform(ppg, processor.fs, 2)
            fft[fft <= 40/60] = 0
            fft[fft >= 240/60] = 0

            pred_fs = predictions[i] / 55
            pred_fs_win = (freqs >= pred_fs - 0.5) & (freqs <= pred_fs + 0.5)
            confs = np.sum(fft[pred_fs_win]) / (np.sum(fft) + 1e-6)
            _confidences.append(confs)
            _errors.append(np.abs(predictions[i] - labels[i]))
        
        errors.append(_errors)
        confidences.append(_confidences)
    
    errors = np.hstack(errors)
    confidences = np.hstack(confidences)
    
    return errors, confidences

In [168]:
def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    
    # Run the pulse rate algorithm on each trial in the dataset
    errors, confidences = RunPulseRateAlgorithm(data_fls, ref_fls)
        
    return AggregateErrorMetric(errors, confidences)

In [169]:
Evaluate()

8.043776496326208

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

### Answers

> - **Code Description** - The structure of the notebook is as follows:
>   - *Troika Dataset Functions*: these blocks contain the functions in charge of loading the dataset data.
>   - *Auxiliar Functions*: these blocks contain common functions throughout the project.
>   - *Classes*: these blocks contain the two main classes that englobe the project. The first one, DataProcessor, is in class in charge of handling the dataset samples and processing them. The second one, Estimator, is the one containing the model and the interfaces to use it for training and predicting.
>   - *Main functions*: finally, these blocks contain the top-level functions running the code.
> - **Data Description** - The dataset is composed of PPG and accelerator signals as well as BPMs for each of the samples present in the data. 
> - **Algorithhm Description** will include the following:
>   - how the algorithm works: the data is first process into eight second windows in the range of 40 to 240 bpm with a separation of two seconds between each of them. The three channels of the accelerometer are combined to get the magnitude of the accelerometer itself to later be processed into three features along the three more extracted from the PPG.
>   - the specific aspects of the physiology that it takes advantage of: the PPG values are obtain by emitting light into the wrists which is absorbed by the red blood cells present in the veins. Every time the veins are fulfill with blood, these causes an increase in the amount of blood cells, absorbing more light. On the other hand, when the blood pressure decreases, less light is absorbed due to the decrease of blood cells' presence. Moreover, the accelerometer signals keep track of the movement of the wrists, giving information about what is going at a physical level to contrast with any rare increase or decrease in the blood pressure apart from the heart beats.
>   - a describtion of the algorithm outputs: the algorithm returns the estimated frequency and the confidence of the prediction.
>   - caveats on algorithm outputs: the confidence is computed by adding up the magnitudes of the frequencies where its value was close to the model's estimation and dividing them by the total magnitude.
>   - common failure modes: any change in the PPG due to a wrist, arm or finger movement.
> - **Algorithm Performance** - The main metric used to measure the model's performance was the Mean Absolute Error (MAE) between the heart rate estimated and the expected one. The final layer of the evaluation takes this metrics and computes the MAE at 90% availability, giving a result of `8.044`. It is important to consider that the dataset was composed of only 12 people, which is a low amount of individual samples and the results could be biased to any statistical deviation present in this group.

-----
### Next Steps
You will now go to **Test Your Algorithm** (back in the Project Classroom) to apply a unit test to confirm that your algorithm met the success criteria. 